# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
# Import API key
from api_keys import api_google

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import the csv file I created from PyWeather
city_df = pd.read_csv("../PyWeather/city_df.csv")
city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kindu,-2.95,25.95,70.50,93,98,1.32,CD,1606238556
1,Dauphin,51.15,-100.05,26.60,92,90,14.99,CA,1606238557
2,Saint-Philippe,-21.36,55.77,75.20,78,53,8.99,RE,1606238558
3,Port Hardy,50.70,-127.42,42.80,100,90,4.70,CA,1606238558
4,San Quintín,30.48,-115.95,61.52,68,3,3.67,MX,1606238558
...,...,...,...,...,...,...,...,...,...
542,Oodweyne,9.41,45.06,70.95,73,98,9.89,SO,1606238822
543,Luwuk,-0.95,122.79,78.67,73,61,3.02,ID,1606238777
544,Manadhoo,5.77,73.38,82.17,67,26,7.18,MV,1606238823
545,Onguday,50.75,86.15,18.32,91,100,0.02,RU,1606238823


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Import the API key to properly display figure
gmaps.configure(api_key=api_google)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Create the data frame for my ideal vacation spot
# Use the loc function to locate the cities with ideal spots
ideal_weather_df = city_df.loc[(city_df["Max Temp"]>50) & (city_df["Max Temp"]<80) & (city_df["Cloudiness"]>80)]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kindu,-2.95,25.95,70.50,93,98,1.32,CD,1606238556
9,Rikitea,-23.12,-134.97,73.87,76,100,12.41,PF,1606238561
41,Antalaha,-14.90,50.28,74.57,83,84,8.10,MG,1606238576
44,Torbay,47.67,-52.73,54.00,100,90,40.26,CA,1606238578
57,Kapaa,22.08,-159.32,71.60,94,90,24.16,US,1606238584


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Set up the hotel data frame
hotel_df = ideal_weather_df
hotel_df["Hotel Name"] = ""
hotel_df = hotel_df[["City", "Lat", "Lng", "Hotel Name", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Country"]]
hotel_df.head()

,City,Lat,Lng,Hotel Name,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Kindu,-2.95,25.95,,70.50,93,98,1.32,CD
9,Rikitea,-23.12,-134.97,,73.87,76,100,12.41,PF
41,Antalaha,-14.90,50.28,,74.57,83,84,8.10,MG
44,Torbay,47.67,-52.73,,54.00,100,90,40.26,CA
57,Kapaa,22.08,-159.32,,71.60,94,90,24.16,US


In [20]:
# Set up the parameters to search for hotels based on lat/long values using iterrows
for index,city in hotel_df.iterrows():

    coordinates = f"{city['Lat']},{city['Lng']}"

    params = {
    "location": coordinates,
    "radius": 5000,
    "type": "lodging",
    "key": api_google
    }

# https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&key=YOUR_API_KEY
# Create the base url and request the json from the url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    response = response.json()
    #pprint(response)
    
# Go into the json file and pull out the 'name' of the hotel and place into the df
    try:
        hotel_name = response["results"][0]["name"]
        hotel_df.loc[index, 'Hotel Name'] = hotel_name
        
    except:
        print("skip")

skip
skip
skip
skip
skip


In [24]:
# Check to see if the hotel name printed out in data frame
hotel_df.head(20)
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
hotel_df

,City,Lat,Lng,Hotel Name,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Kindu,-2.95,25.95,Résidence Okanda,70.50,93,98,1.32,CD
9,Rikitea,-23.12,-134.97,Pension Maro'i,73.87,76,100,12.41,PF
41,Antalaha,-14.90,50.28,Hotel Vitasoa,74.57,83,84,8.10,MG
44,Torbay,47.67,-52.73,Gallows Cove,54.00,100,90,40.26,CA
57,Kapaa,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach,71.60,94,90,24.16,US
...,...,...,...,...,...,...,...,...,...
511,Wewak,-3.55,143.63,In Wewak Boutique Hotel,75.81,89,100,3.47,PG
523,Sakib,32.28,35.81,Saleh Kayyali's Farm,59.00,47,98,8.05,JO
526,Peniche,39.36,-9.38,Hotel Star inn Peniche,61.00,79,89,11.99,PT
534,Cairns,-16.92,145.77,Mantra Trilogy Cairns,75.20,83,90,3.67,AU


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))